In [1]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

model = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")
print("模型加载完成")

vector_store = Chroma(
    persist_directory="data/chroma",
    embedding_function=model,
    collection_name="postgis-3.6-en",
)
print("向量数据库加载完成")

模型加载完成
向量数据库加载完成


In [10]:
similar_docs = vector_store.similarity_search(
    "A TIN is a collection of non-overlapping Triangles representing a Triangulated Irregular Network.", 
    k=2)
print(len(similar_docs))
for doc in similar_docs:
    print(doc.page_content)

2
((0 0 0, 1 0 0, 1 0 1, 0 0 1, 0 0 0)),
((1 1 0, 1 1 1, 1 0 1, 1 0 0, 1 1 0)),
((0 1 0, 0 1 1, 1 1 1, 1 1 0, 0 1 0)),
((0 0 1, 1 0 1, 1 1 1, 0 1 1, 0 0 1)) )
4.1.1.10 Triangle
A Triangle is a polygon deﬁned by three distinct non-collinear vertices. Because a Triangle is a polygon it is speciﬁed by four
coordinates, with the ﬁrst and fourth being equal.
TRIANGLE ((0 0, 0 9, 9 0, 0 0))
4.1.1.11 TIN
A TIN is a collection of non-overlapping Triangles representing a Triangulated Irregular Network.
A TIN is a collection of non-overlapping Triangles representing a Triangulated Irregular Network.
TIN Z ( ((0 0 0, 0 0 1, 0 1 0, 0 0 0)), ((0 0 0, 0 1 0, 1 1 0, 0 0 0)) )
4.1.2 SQL/MM Part 3 - Curves
The ISO/IEC 13249-3 SQL Multimedia - Spatial standard (SQL/MM) extends the OGC SFA to deﬁne Geometry subtypes con-
taining curves with circular arcs. The SQL/MM types support 3DM, 3DZ and 4D coordinates.
Note


In [18]:
from langchain_core.documents import Document

new_docs = [Document(page_content="my name is ashlii.", metadata={"source": "custom"})]

vector_store.add_documents(new_docs)

['235628aa-94f9-48fb-a801-1b1bfd0d923a']

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

In [20]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [21]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a postgis document. "
    "Use the tool to help answer user queries."
)
agent = create_agent(llm, tools, system_prompt=prompt)

In [24]:
query = (
    "what is my name? look for the answer in the retrieved context."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is my name? look for the answer in the retrieved context.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_cCQzxnBH0xzpDvEgQRoQBvij)
 Call ID: call_cCQzxnBH0xzpDvEgQRoQBvij
  Args:
    query: what is my name?
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'custom'}
Content: my name is ashlii.

Source: {'producer': 'pdfTeX-1.40.21', 'total_pages': 946, 'keywords': '', 'page': 94, 'subject': '', 'creator': 'DBLaTeX-0.3.12', 'trapped': '/False', 'creationdate': '2025-10-15T21:40:24+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020/Debian) kpathsea version 6.3.2', 'moddate': '2025-10-15T21:40:24+00:00', 'source': 'data/postgis-3.6-en.pdf', 'page_label': '68', 'title': 'PostGIS 3.6.1dev Manual', 'author': ''}
Content: Cast

In [26]:
vector_store.delete(['235628aa-94f9-48fb-a801-1b1bfd0d923a'])